# Virtual screening of bioassay data

Drug discovery is the first stage of the drug-development process and involves finding compounds to test and screen against biological targets. This first stage is known as primary-screening and usually involves the screening of thousands of compounds.

The bioassay data is a collection of 21 bioassays (screens) that measure the activity of various compounds against different biological targets. This project is mostly based on the [original study](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2820499/) of the data.

![Image of Yaktocat](https://i.imgur.com/2bKnxBf.jpg)

In [1]:
import pandas as pd
import numpy as np
import itertools
import os

# Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

pd.options.display.max_columns = None

import warnings
warnings.simplefilter(action = "ignore")

### Loading and cleaning the data

Each bioassay is split into test and train datasets. Since there are multiple datasets, we will load each to the variables "train" and "test" respectively.

In [2]:
def load_X(X_signals_paths):
    X_signals = []
    for signal_type_path in X_signals_paths:
        file = pd.read_csv(signal_type_path)
        X_signals.append(file)
    return X_signals 

In [3]:
DATA_PATH_train = "train/"
data_paths_train = os.listdir(DATA_PATH_train)
datasets_train = [DATA_PATH_train + signal for signal in data_paths_train]
train = load_X(datasets_train)

DATA_PATH_test = "test/"
data_paths_test = os.listdir(DATA_PATH_test)
datasets_test = [DATA_PATH_test + signal for signal in data_paths_test]
test = load_X(datasets_test)

In [4]:
print("There are {} train files located at: ".format(len(train)) + DATA_PATH_train + "\n\n" + "Files in directory:")
for i in os.listdir(DATA_PATH_train):
    print(i)

There are 21 train files located at: train/

Files in directory:
AID1284Morered_train.csv
AID1284red_train.csv
AID1608Morered_train.csv
AID1608red_train.csv
AID362red_train.csv
AID373AID439red_train.csv
AID373red_train.csv
AID439Morered_train.csv
AID439red_train.csv
AID456red_train.csv
AID604AID644_AllRed_train.csv
AID604red_train.csv
AID644Morered_train.csv
AID644red_train.csv
AID687AID721red_train.csv
AID687red_train.csv
AID688red_train.csv
AID721morered_train.csv
AID721red_train.csv
AID746AID1284red_train.csv
AID746red_train.csv


In [5]:
print("There are {} test files located at: ".format(len(test)) + DATA_PATH_test + "\n\n" + "Files in directory:")
for i in os.listdir(DATA_PATH_test):
    print(i)

There are 21 test files located at: test/

Files in directory:
AID1284Morered_test.csv
AID1284red_test.csv
AID1608Morered_test.csv
AID1608red_test.csv
AID362red_test.csv
AID373AID439red_test.csv
AID373red_test.csv
AID439Morered_test.csv
AID439red_test.csv
AID456red_test.csv
AID604AID644_AllRed_test.csv
AID604red_test.csv
AID644Morered_test.csv
AID644red_test.csv
AID687AID721red_test.csv
AID687red_test.csv
AID688red_test.csv
AID721morered_test.csv
AID721red_test.csv
AID746AID1284red_test.csv
AID746red_test.csv


Each dataset has a different number of columns (attributes) and compounds. All of them have an "Outcome" column which describes the activity of the compound. This column can have 3 unique values: active, inactive an inconclusive. For simplicity's sake, the inconclusive compounds will be marked as inactive in this project. 

Let's view the characteristics of each dataset by creating a table below.

In [6]:
# Combining train and test datasets
data = []

for i in range(len(train)):
    data.append(pd.concat([train[i], test[i]]))

In [7]:
column_names = [k[:-10] for k in data_paths_train]
n_active,n_rows,n_columns  = [],[],[]

for i in range(len(data)):
    n_active.append((data[i]['Outcome'].value_counts()[1]/data[i].shape[0])*100)
    n_rows.append(data[i].shape[0])
    n_columns.append(data[i].shape[1])
table_n_active = pd.DataFrame([n_columns, n_rows, n_active], 
                              columns = column_names, 
                              index = ['Number of attributes', 'Number of compounds', '% Active compounds'])

In [8]:
Primary = ['AID362red',
'AID604red',
'AID456red',
'AID688red',
'AID373red',
'AID746red',
'AID687red']
Primary_and_Confirmatory = ['AID746AID1284red',
'AID604AID644_AllRed',
'AID373AID439red',
'AID687AID721red']
Confirmatory = ['AID1608red',
'AID644red',
'AID1284red',
'AID439Morered',
'AID721morered',
'AID1608Morered',
'AID644Morered',
'AID1284Morered',
'AID439red',
'AID721red']

In [9]:
Primary_table = table_n_active[Primary]
PrimCom_table = table_n_active[Primary_and_Confirmatory]
Com_table = table_n_active[Confirmatory]

Primary_table.columns = pd.MultiIndex.from_tuples([('Primary', i) for i in Primary])
PrimCom_table.columns = pd.MultiIndex.from_tuples([('Primary and Confirmatory', i) for i in Primary_and_Confirmatory])
Com_table.columns = pd.MultiIndex.from_tuples([('Confirmatory', i) for i in Confirmatory])

In [10]:
pd.concat([Primary_table, PrimCom_table, Com_table], axis=1).T

Number of attributes  \
Primary                  AID362red                           145.0   
                         AID604red                           155.0   
                         AID456red                           154.0   
                         AID688red                           154.0   
                         AID373red                           156.0   
                         AID746red                           155.0   
                         AID687red                           154.0   
Primary and Confirmatory AID746AID1284red                    155.0   
                         AID604AID644_AllRed                 155.0   
                         AID373AID439red                     155.0   
                         AID687AID721red                     154.0   
Confirmatory             AID1608red                          155.0   
                         AID644red                           101.0   
                         AID1284red                          104.0   
                         AID439Morered                       915.0   
                         AID721morered                       915.0   
                         AID1608Morered                      915.0   
                         AID644Morered                       915.0   
                         AID1284Morered                      915.0   
                         AID439red                            82.0   
                         AID721red                            88.0   

                                              Number of compounds  \
Primary                  AID362red                         4279.0   
                         AID604red                        59788.0   
                         AID456red                         9982.0   
                         AID688red                        27189.0   
                         AID373red                        59788.0   
                         AID746red                        59788.0   
                         AID687red                        33067.0   
Primary and Confirmatory AID746AID1284red                 59784.0   
                         AID604AID644_AllRed              59782.0   
                         AID373AID439red                  59795.0   
                         AID687AID721red                  33067.0   
Confirmatory             AID1608red                        1033.0   
                         AID644red                          206.0   
                         AID1284red                         362.0   
                         AID439Morered                       69.0   
                         AID721morered                       94.0   
                         AID1608Morered                    1033.0   
                         AID644Morered                      206.0   
                         AID1284Morered                     362.0   
                         AID439red                           69.0   
                         AID721red                           94.0   

                                              % Active compounds  
Primary                  AID362red                      1.402197  
                         AID604red                      0.354586  
                         AID456red                      0.270487  
                         AID688red                      0.912134  
                         AID373red                      0.103700  
                         AID746red                      0.612163  
                         AID687red                      0.284271  
Primary and Confirmatory AID746AID1284red               0.095343  
                         AID604AID644_AllRed            0.112074  
                         AID373AID439red                0.021741  
                         AID687AID721red                0.063507  
Confirmatory             AID1608red                     6.582769  
                         AID644red                     32.524272  
                         AID1284red             

The table shows the description of the assays as well as the number of compounds, attributes and the percentage of active compounds. 

From above we notice that the data is unbalanced, with some cases where the active compounds are less than one percent. This means that standard machine learning techniques, which assume equality, are not very effective at building predictive models when there is a low minority class ratio.

In [11]:
# Renaming "Inconclusive" as "Inactive" and replacing the Active and Inactive values with 1's and 0's respectively

mapping = {'Inactive': 0, 'Active': 1}
for i in range(len(train)):
    train[i]['Outcome'] = train[i]['Outcome'].replace(['Inconc', 'Inconclusive'], 'Inactive')
    train[i]['Outcome'] = train[i]['Outcome'].map(mapping)
for i in range(len(test)):
    test[i]['Outcome'] = test[i]['Outcome'].replace(['Inconc', 'Inconclusive'], 'Inactive')
    test[i]['Outcome'] = test[i]['Outcome'].map(mapping)

### Creating models with regular sets

In [12]:
# Train datasets
x, y = [], []
for i in range(len(train)):
    x.append(train[i].drop("Outcome", axis=1))
    y.append(train[i]["Outcome"])

In [13]:
# Test datasets
X_t, y_t = [], []
for i in range(len(test)):
    X_t.append(test[i].drop("Outcome", axis=1))
    y_t.append(test[i]["Outcome"])

In [14]:
# Algorithms
models = []
models.append(('LR', LogisticRegression(random_state=43, class_weight = 'balanced')))
models.append(('CART', DecisionTreeClassifier(random_state=43, class_weight = 'balanced')))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(random_state=43, class_weight = 'balanced')))
models.append(('RFC', RandomForestClassifier(random_state=43, class_weight = 'balanced')))
models.append(('XGB', XGBClassifier()))

For evaluation of the models' performance we will use accuracy, recall and F1 scores:


$$accuracy = \frac{TP + TN}{TP + FN + TN + FP}$$

$$recall = \frac{TP}{TP + FN}$$

$$F1 = \frac{2 * precision * recall}{precision + recall}$$

where $TP$ is the number of True Positives, $TN$ is the number of True Negatives, $FN$ is the number of False Negatives and $FP$ is the number of False Positives.

In [15]:
def ml_testing(x,y,xt,yt,t1):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    pred_results = []
    recall = []
    f1 = []
    names = []
    for name, model in models:
        # Creating models
        fit_train = model.fit(x_train,y_train)
        names.append(name)
        table1 = pd.DataFrame(index=names)
        # Testing models on the test data
        y_result = model.predict(xt)
        pred_results.append(accuracy_score(yt, y_result))
        table1['Accuracy'] = pred_results
        # Recall scores
        recall.append(recall_score(yt, y_result, labels=[1, 0]))
        table1['Recall'] = recall
        # F1 scores
        f1.append(f1_score(yt, y_result, labels=[1, 0]))
        table1['F1'] = f1
    t1.append(table1)

In [16]:
DF = []
for i in range(len(test)):
    ml_testing(x[i], y[i], X_t[i], y_t[i], DF)

In [17]:
for i in range(len(column_names)):
    DF[i].columns = pd.MultiIndex.from_tuples([(column_names[i],'Accuracy'),
                                               (column_names[i],'Recall'),
                                               (column_names[i],'F1')])

In [18]:
DF=pd.concat(DF, axis=1)

#### Results

In [19]:
# Primary screening assays
DF[Primary].T

LR      CART        NB       SVC       RFC       XGB
AID362red Accuracy  0.889019  0.983645  0.248832  0.968458  0.987150  0.985981
          Recall    0.583333  0.250000  0.833333  0.250000  0.166667  0.000000
          F1        0.128440  0.300000  0.030166  0.181818  0.266667  0.000000
AID604red Accuracy  0.768337  0.993477  0.189763  0.975245  0.996487  0.996487
          Recall    0.666667  0.047619  0.976190  0.119048  0.000000  0.000000
          F1        0.019816  0.048780  0.008393  0.032680  0.000000  0.000000
AID456red Accuracy  0.940882  0.994990  0.427856  0.991984  0.997495  0.997495
          Recall    0.800000  0.000000  0.600000  0.000000  0.000000  0.000000
          F1        0.063492  0.000000  0.005226  0.000000  0.000000  0.000000
AID688red Accuracy  0.684994  0.975910  0.099669  0.942810  0.990622  0.990805
          Recall    0.280000  0.020000  0.920000  0.040000  0.000000  0.000000
          F1        0.016083  0.015038  0.018444  0.012698  0.000000  0.000000
AID373red Accuracy  0.888350  0.998244  0.372752  0.996069  0.999080  0.998913
          Recall    0.583333  0.166667  0.750000  0.000000  0.083333  0.000000
          F1        0.010378  0.160000  0.002394  0.000000  0.153846  0.000000
AID746red Accuracy  0.723091  0.989295  0.150958  0.955173  0.994062  0.993978
          Recall    0.684932  0.150685  0.917808  0.205479  0.027397  0.013699
          F1        0.029317  0.146667  0.013027  0.053004  0.053333  0.027027
AID687red Accuracy  0.826403  0.993346  0.217904  0.988356  0.997278  0.997127
          Recall    0.277778  0.000000  0.944444  0.000000  0.000000  0.000000
          F1        0.008636  0.000000  0.006531  0.000000  0.000000  0.000000

In [20]:
# Primary and Confirmatory screening assays
DF[Primary_and_Confirmatory].T

LR      CART        NB       SVC  \
AID746AID1284red    Accuracy  0.885079  0.998244  0.344931  0.997742   
                    Recall    0.727273  0.090909  0.909091  0.272727   
                    F1        0.011511  0.086957  0.002547  0.181818   
AID604AID644_AllRed Accuracy  0.872867  0.998076  0.433172  0.995316   
                    Recall    0.692308  0.153846  0.846154  0.153846   
                    F1        0.011704  0.148148  0.003236  0.066667   
AID373AID439red     Accuracy  0.994481  0.999833  0.772222  0.999331   
                    Recall    0.000000  0.500000  0.000000  0.000000   
                    F1        0.000000  0.500000  0.000000  0.000000   
AID687AID721red     Accuracy  0.902011  0.998185  0.425072  0.998034   
                    Recall    0.000000  0.000000  0.750000  0.000000   
                    F1        0.000000  0.000000  0.001576  0.000000   

                                   RFC       XGB  
AID746AID1284red    Accuracy  0.999080  0.999080  
                    Recall    0.090909  0.000000  
                    F1        0.153846  0.000000  
AID604AID644_AllRed Accuracy  0.998913  0.998913  
                    Recall    0.000000  0.000000  
                    F1        0.000000  0.000000  
AID373AID439red     Accuracy  0.999833  0.999833  
                    Recall    0.000000  0.000000  
                    F1        0.000000  0.000000  
AID687AID721red     Accuracy  0.999395  0.999395  
                    Recall    0.000000  0.000000  
                    F1        0.000000  0.000000

In [21]:
# Confirmatory screening assays
DF[Confirmatory].T

LR      CART        NB       SVC       RFC  \
AID1608red     Accuracy  0.825243  0.868932  0.266990  0.878641  0.936893   
               Recall    0.230769  0.076923  0.769231  0.153846  0.000000   
               F1        0.142857  0.068966  0.116959  0.137931  0.000000   
AID644red      Accuracy  0.536585  0.658537  0.463415  0.609756  0.634146   
               Recall    0.307692  0.230769  0.769231  0.230769  0.076923   
               F1        0.296296  0.300000  0.476190  0.272727  0.117647   
AID1284red     Accuracy  0.722222  0.736111  0.305556  0.777778  0.861111   
               Recall    0.545455  0.181818  1.000000  0.454545  0.181818   
               F1        0.375000  0.173913  0.305556  0.384615  0.285714   
AID439Morered  Accuracy  0.692308  0.923077  0.769231  0.615385  0.769231   
               Recall    0.500000  0.500000  0.500000  1.000000  0.000000   
               F1        0.333333  0.666667  0.400000  0.444444  0.000000   
AID721morered  Accuracy  0.722222  0.500000  0.611111  0.555556  0.722222   
               Recall    0.250000  0.000000  0.000000  0.500000  0.000000   
               F1        0.285714  0.000000  0.000000  0.333333  0.000000   
AID1608Morered Accuracy  0.893204  0.888350  0.669903  0.660194  0.932039   
               Recall    0.307692  0.076923  0.307692  0.230769  0.076923   
               F1        0.266667  0.080000  0.105263  0.078947  0.125000   
AID644Morered  Accuracy  0.658537  0.609756  0.609756  0.609756  0.731707   
               Recall    0.384615  0.230769  0.461538  0.461538  0.307692   
               F1        0.416667  0.272727  0.428571  0.428571  0.421053   
AID1284Morered Accuracy  0.791667  0.791667  0.708333  0.652778  0.861111   
               Recall    0.545455  0.272727  0.727273  0.454545  0.181818   
               F1        0.444444  0.285714  0.432432  0.285714  0.285714   
AID439red      Accuracy  0.692308  0.769231  0.307692  0.846154  0.846154   
               Recall    0.500000  0.500000  1.000000  0.000000  0.000000   
               F1        0.333333  0.400000  0.307692  0.000000  0.000000   
AID721red      Accuracy  0.722222  0.666667  0.555556  0.722222  0.722222   
               Recall    0.500000  0.000000  0.250000  0.000000  0.000000   
               F1        0.444444  0.000000  0.200000  0.000000  0.000000   

                              XGB  
AID1608red     Accuracy  0.936893  
               Recall    0.000000  
               F1        0.000000  
AID644red      Accuracy  0.634146  
               Recall    0.076923  
               F1        0.117647  
AID1284red     Accuracy  0.833333  
               Recall    0.181818  
               F1        0.250000  
AID439Morered  Accuracy  0.846154  
               Recall    0.000000  
               F1        0.000000  
AID721morered  Accuracy  0.500000  
               Recall    0.000000  
               F1        0.000000  
AID1608Morered Accuracy  0.932039  
               Recall    0.000000  
               F1        0.000000  
AID644Morered  Accuracy  0.658537  
               Recall    0.153846  
               F1        0.222222  
AID1284Morered Accuracy  0.847222  
               Recall    0.272727  
               F1        0.352941  
AID439red      Accuracy  0.846154  
               Recall    0.000000  
               F1        0.000000  
AID721red      Accuracy  0.500000  
               Recall    0.000000  
               F1        0.000000

A lot of the classifiers have relatively high accuracy scores, but accuracy applies a naive 0.50 threshold to decide between classes, and this is usually wrong when the classes are imbalanced. For instance, the classifier may assume that all outcomes are inactive and get a high accuracy score, but our goal here is to predict which compounds are active.

We get a better understanding of the classifier's performance by reviewing its recall and F1 score.
The bad results indicate that as expected, the quality of the data is poor and that maybe we should try alternative way to improve the models' performances.

### Creating models with undersampled data

Undersampling randomly downsamples the majority class. Below, we will create a ratio of 1:1 between the active and inactive compounds by randomly removing portions of the inactive compounds. This may improve the performance of the classifiers.

In [22]:
active = []
inactive = []
for i in range(len(train)):
    actives = len(train[i][train[i]['Outcome'] == 1])
    active.append(actives)
    inactives = train[i][train[i].Outcome == 0].index
    inactive.append(inactives)

In [23]:
random_indices = []
for a, i in zip(active, inactive):
    random_indice = np.random.choice(i, a, replace=False)
    random_indices.append(random_indice)

In [24]:
active_indices = []
for i in range(len(train)):
    active_indice = train[i][train[i].Outcome == 1].index
    active_indices.append(active_indice)

In [25]:
under_sample_indices = []
for i in range(len(random_indices)):
    under_sample_indice = np.concatenate([active_indices[i], random_indices[i]])
    under_sample_indices.append(under_sample_indice)

In [26]:
under_samples = []
for i in range(len(under_sample_indices)):
    under_sample = train[i].iloc[under_sample_indices[i]]
    under_samples.append(under_sample)

In [27]:
x, y = [], []
for i in range(len(under_samples)):
    x.append(under_samples[i].drop("Outcome", axis=1))
    y.append(under_samples[i]["Outcome"])

In [28]:
DF_undersampled = []
for i in range(len(test)):
    ml_testing(x[i],y[i],X_t[i],y_t[i], DF_undersampled)

In [29]:
for i in range(len(column_names)):
    DF_undersampled[i].columns = pd.MultiIndex.from_tuples([(column_names[i],'Accuracy'),
                                                            (column_names[i],'Recall'),
                                                            (column_names[i],'F1')])

In [30]:
DF_undersampled=pd.concat(DF_undersampled, axis=1)

#### Results

In [31]:
# Primary screening assays
DF_undersampled[Primary].T

LR      CART        NB       SVC       RFC       XGB
AID362red Accuracy  0.679907  0.595794  0.823598  0.648364  0.739486  0.721963
          Recall    1.000000  0.500000  0.750000  0.750000  0.750000  0.750000
          F1        0.080537  0.033520  0.106509  0.056426  0.074689  0.070312
AID604red Accuracy  0.678682  0.644225  0.218449  0.670402  0.768504  0.696998
          Recall    0.809524  0.547619  1.000000  0.619048  0.666667  0.761905
          F1        0.017391  0.010698  0.008909  0.013023  0.019830  0.017358
AID456red Accuracy  0.646293  0.416333  0.749499  0.295090  0.700401  0.560120
          Recall    1.000000  0.800000  0.600000  1.000000  0.400000  0.800000
          F1        0.013966  0.006820  0.011858  0.007057  0.006645  0.009029
AID688red Accuracy  0.562155  0.534939  0.105554  0.452924  0.655940  0.568775
          Recall    0.500000  0.580000  0.920000  0.700000  0.460000  0.460000
          F1        0.020568  0.022420  0.018563  0.022989  0.023996  0.019239
AID373red Accuracy  0.719495  0.693401  0.368822  0.665886  0.838672  0.765493
          Recall    0.916667  0.750000  0.833333  0.500000  0.833333  0.916667
          F1        0.006517  0.004886  0.002643  0.002995  0.010262  0.007785
AID746red Accuracy  0.649327  0.618132  0.165928  0.557999  0.775529  0.715480
          Recall    0.698630  0.712329  0.890411  0.479452  0.671233  0.643836
          F1        0.023749  0.022270  0.012867  0.013072  0.035226  0.026888
AID687red Accuracy  0.576440  0.576440  0.179495  0.607137  0.746862  0.645698
          Recall    0.666667  0.333333  1.000000  0.333333  0.277778  0.277778
          F1        0.008496  0.004266  0.006591  0.004598  0.005938  0.004250

In [32]:
# Primary and Confirmatory screening assays
DF_undersampled[Primary_and_Confirmatory].T

LR      CART        NB       SVC  \
AID746AID1284red    Accuracy  0.606139  0.567581  0.235112  0.688776   
                    Recall    1.000000  0.909091  1.000000  0.454545   
                    F1        0.004650  0.003854  0.002400  0.002680   
AID604AID644_AllRed Accuracy  0.678237  0.647792  0.355888  0.722650   
                    Recall    0.846154  0.538462  1.000000  0.615385   
                    F1        0.005686  0.003314  0.003365  0.004802   
AID373AID439red     Accuracy  0.697717  0.616941  0.510578  0.775985   
                    Recall    0.500000  1.000000  1.000000  1.000000   
                    F1        0.000553  0.000872  0.000683  0.001491   
AID687AID721red     Accuracy  0.493120  0.621654  0.482988  0.616362   
                    Recall    0.750000  0.750000  0.500000  0.250000   
                    F1        0.001787  0.002392  0.001169  0.000788   

                                   RFC       XGB  
AID746AID1284red    Accuracy  0.634242  0.604466  
                    Recall    0.727273  0.909091  
                    F1        0.003645  0.004211  
AID604AID644_AllRed Accuracy  0.778019  0.732687  
                    Recall    0.538462  0.692308  
                    F1        0.005247  0.005600  
AID373AID439red     Accuracy  0.714943  0.669621  
                    Recall    0.500000  0.500000  
                    F1        0.000586  0.000506  
AID687AID721red     Accuracy  0.566460  0.442462  
                    Recall    0.750000  0.750000  
                    F1        0.002088  0.001625

In [33]:
# Confirmatory screening assays
DF_undersampled[Confirmatory].T

LR      CART        NB       SVC       RFC  \
AID1608red     Accuracy  0.558252  0.432039  0.567961  0.577670  0.592233   
               Recall    0.538462  0.538462  0.461538  0.307692  0.461538   
               F1        0.133333  0.106870  0.118812  0.084211  0.125000   
AID644red      Accuracy  0.536585  0.487805  0.609756  0.560976  0.609756   
               Recall    0.384615  0.384615  0.153846  0.461538  0.307692   
               F1        0.344828  0.322581  0.200000  0.400000  0.333333   
AID1284red     Accuracy  0.555556  0.444444  0.319444  0.569444  0.583333   
               Recall    0.818182  0.636364  1.000000  0.545455  0.545455   
               F1        0.360000  0.259259  0.309859  0.279070  0.285714   
AID439Morered  Accuracy  0.846154  0.615385  0.769231  0.538462  0.692308   
               Recall    0.500000  0.000000  0.500000  1.000000  0.500000   
               F1        0.500000  0.000000  0.400000  0.400000  0.333333   
AID721morered  Accuracy  0.500000  0.500000  0.555556  0.388889  0.555556   
               Recall    0.250000  0.000000  0.250000  0.500000  0.000000   
               F1        0.181818  0.000000  0.200000  0.266667  0.000000   
AID1608Morered Accuracy  0.611650  0.538835  0.451456  0.626214  0.718447   
               Recall    0.461538  0.307692  0.538462  0.307692  0.461538   
               F1        0.130435  0.077670  0.110236  0.094118  0.171429   
AID644Morered  Accuracy  0.536585  0.536585  0.512195  0.317073  0.609756   
               Recall    0.461538  0.384615  0.615385  0.538462  0.461538   
               F1        0.387097  0.344828  0.444444  0.333333  0.428571   
AID1284Morered Accuracy  0.652778  0.597222  0.444444  0.569444  0.583333   
               Recall    0.818182  0.636364  0.909091  0.636364  0.727273   
               F1        0.418605  0.325581  0.333333  0.311111  0.347826   
AID439red      Accuracy  0.923077  0.615385  0.615385  0.923077  0.692308   
               Recall    1.000000  0.500000  1.000000  0.500000  1.000000   
               F1        0.800000  0.285714  0.444444  0.666667  0.500000   
AID721red      Accuracy  0.555556  0.611111  0.666667  0.277778  0.333333   
               Recall    0.750000  0.750000  0.750000  0.750000  0.750000   
               F1        0.428571  0.461538  0.500000  0.315789  0.333333   

                              XGB  
AID1608red     Accuracy  0.466019  
               Recall    0.538462  
               F1        0.112903  
AID644red      Accuracy  0.512195  
               Recall    0.307692  
               F1        0.285714  
AID1284red     Accuracy  0.527778  
               Recall    0.636364  
               F1        0.291667  
AID439Morered  Accuracy  0.615385  
               Recall    0.500000  
               F1        0.285714  
AID721morered  Accuracy  0.388889  
               Recall    0.000000  
               F1        0.000000  
AID1608Morered Accuracy  0.480583  
               Recall    0.461538  
               F1        0.100840  
AID644Morered  Accuracy  0.487805  
               Recall    0.384615  
               F1        0.322581  
AID1284Morered Accuracy  0.569444  
               Recall    0.727273  
               F1        0.340426  
AID439red      Accuracy  0.615385  
               Recall    0.500000  
               F1        0.285714  
AID721red      Accuracy  0.388889  
               Recall    0.750000  
               F1        0.352941

The performance was improved in most of the cases, but not by a large margin.

### Hyperparameter XGBoost optimisation

Hyperparameter optimization or tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm. 
Since tuning every classifier for every bioassay would require a lot of time and computational power, let's examine a single example.

#### Part 1: Untuned model parameters 

In [34]:
train = pd.read_csv('...AID1284Morered_train.csv')
test = pd.read_csv('...AID1284Morered_test.csv')

In [35]:
mapping = {'Inactive': 0, 'Active': 1}
train['Outcome'] = train['Outcome'].map(mapping)
test['Outcome'] = test['Outcome'].map(mapping)

In [36]:
x = train.drop("Outcome", axis=1)
y = train['Outcome']

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [38]:
XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [39]:
# Train set
XGB = xgb.XGBClassifier()
XGB.fit(x_train, y_train)
predictions = XGB.predict(x_test)
accuracy_score(y_test, predictions)

0.8045977011494253

In [40]:
# Test set
X_t = test.drop("Outcome", axis=1)
y_result = XGB.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8472222222222222

In [41]:
# Confusion matrix
XGB_confusion_matrix1 = confusion_matrix(test["Outcome"], y_result, labels=[1, 0])
XGB_confusion_matrix1

array([[ 3,  8],
       [ 3, 58]], dtype=int64)

Confusion matrix (from left to right):

* True positive (TP): correct positive prediction
* False negative (FN): incorrect negative prediction
* False positive (FP): incorrect positive prediction
* True negative (TN): correct negative prediction

In [42]:
# Recall and F1 score
XGB_recall1 = recall_score(test["Outcome"], y_result, labels=[1, 0]) 
XGB_F11 = f1_score(test["Outcome"], y_result, labels=[1, 0])

In [43]:
print('Recall score: ' + str(XGB_recall1), '\nF1 score: ' + str(XGB_F11))

Recall score: 0.2727272727272727 
F1 score: 0.3529411764705882


#### Part 2: Tuning

In [44]:
XGB1 = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=1000,
                         max_depth=5, 
                         min_child_weight=1, 
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         objective= 'binary:logistic', 
                         nthread=4,scale_pos_weight=1,seed=27)
XGB1.fit(x_train, y_train)
predictions = XGB1.predict(x_test)
accuracy_score(y_test, predictions)

0.8160919540229885

In [45]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB1.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8611111111111112

In [46]:
# Step 1: Tune max_depth and min_child_weight
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5,
                                                  min_child_weight=1,
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch1.fit(x_train, y_train)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 3, 'min_child_weight': 1}, 0.5652100840336135)

The optimal parameter for max_depth is 3, while the optimal parameter for min_child_weight is 1.

In [47]:
# Fix new parameters with optimal parameters
XGB2 = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=1000,
                         max_depth=3, 
                         min_child_weight=1, 
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         objective= 'binary:logistic', 
                         nthread=4,
                         scale_pos_weight=1,
                         seed=27)
XGB2.fit(x_train, y_train)
predictions = XGB2.predict(x_test)
accuracy_score(y_test, predictions)

0.8045977011494253

In [48]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB2.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8611111111111112

In [49]:
# Step 2: Tune gamma value using the parameters already tuned above.
param_test2 = { 
    'gamma':[i/10.0 for i in range(0,5)] 
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth = 3,
                                                  min_child_weight = 1,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test2, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch2.fit(x_train, y_train)
gsearch2.best_params_, gsearch2.best_score_

({'gamma': 0.4}, 0.5844817927170868)

In [50]:
# Fix new parameter with optimal parameter
XGB3 = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=1000,
                         max_depth=3, 
                         min_child_weight=1, 
                         gamma=0.4,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         objective= 'binary:logistic', 
                         nthread=4,
                         scale_pos_weight=1,
                         seed=27)
XGB3.fit(x_train, y_train)
predictions = XGB3.predict(x_test)
accuracy_score(y_test, predictions)

0.7931034482758621

In [51]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB3.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8472222222222222

In [52]:
#Step 3: Tune subsample and colsample_bytree
param_test3 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=1000, 
                                                  max_depth = 3,
                                                  min_child_weight = 1,
                                                  gamma=0.4, 
                                                  subsample=0.8, colsample_bytree=0.8,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test3, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch3.fit(x_train, y_train)
gsearch3.best_params_, gsearch3.best_score_

({'colsample_bytree': 0.6, 'subsample': 0.9}, 0.6212885154061625)

In [53]:
XGB4 = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=1000,
                         max_depth=3, 
                         min_child_weight=1, 
                         gamma=0.4,
                         subsample=0.9,
                         colsample_bytree=0.6,
                         objective= 'binary:logistic', 
                         nthread=4,
                         scale_pos_weight=1,
                         seed=27)
XGB4.fit(x_train, y_train)
predictions = XGB4.predict(x_test)
accuracy_score(y_test, predictions)

0.7931034482758621

In [54]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB4.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8472222222222222

In [55]:
# Refining the research
param_test4 = {
 'subsample':[i/100.0 for i in range(55,70,5)],
 'colsample_bytree':[i/100.0 for i in range(85,100,5)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth = 3,
                                                  min_child_weight = 5,
                                                  gamma=0.2, 
                                                  subsample=0.9, 
                                                  colsample_bytree=0.6,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test4, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch4.fit(x_train, y_train)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.55}, 0.5757703081232493)

In [56]:
# Fix new param with optimal parameter
XGB5 = xgb.XGBClassifier(learning_rate =0.1, 
                         n_estimators=1000,
                         max_depth=3, 
                         min_child_weight=1, 
                         gamma=0.4,
                         subsample=0.55,
                         colsample_bytree=0.9,
                         objective= 'binary:logistic', 
                         nthread=4,
                         scale_pos_weight=1,
                         seed=27)
XGB5.fit(x_train, y_train)
predictions = XGB5.predict(x_test)
accuracy_score(y_test, predictions)

0.8045977011494253

In [57]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB5.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.875

In [58]:
#Step 3: tuning learning_rate
param_test5 = {
 'learning_rate':[i/1000.0 for i in range(5,20,2)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  gamma=0.4,
                                                  subsample=0.55,
                                                  colsample_bytree=0.85,
                                                  max_depth = 3,
                                                  min_child_weight = 5,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                        param_grid = param_test5, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch5.fit(x_train, y_train)
gsearch5.best_params_, gsearch5.best_score_

({'learning_rate': 0.019}, 0.5797759103641457)

In [59]:
# Final model
XGB6 = xgb.XGBClassifier(learning_rate =0.019, 
                         n_estimators=1000,
                         max_depth=3, 
                         min_child_weight=1, 
                         gamma=0.4,
                         subsample=0.55,
                         colsample_bytree=0.9,
                         objective= 'binary:logistic', 
                         nthread=4,
                         scale_pos_weight=1,
                         seed=27)
XGB6.fit(x_train, y_train)
predictions = XGB6.predict(x_test)
accuracy_score(y_test, predictions)

0.8045977011494253

In [60]:
X_t = test.drop("Outcome", axis=1)
y_result = XGB6.predict(X_t)
accuracy_score(test["Outcome"], y_result)

0.8611111111111112

Recall and F1 scores:

In [61]:
XGB_recall2 = recall_score(test["Outcome"], y_result, labels=[1, 0])
XGB_F12 = f1_score(test["Outcome"], y_result, labels=[1, 0])

In [62]:
print('Old recall score: ' + str(XGB_recall1), '\nNew recall score: ' + str(XGB_recall2))

Old recall score: 0.2727272727272727 
New recall score: 0.36363636363636365


In [63]:
print('Old F1 score: ' + str(XGB_F11), '\nNew F1 score: ' + str(XGB_F12))

Old F1 score: 0.3529411764705882 
New F1 score: 0.4444444444444444


Both the recall and F1 score improved.

In [64]:
# Confusion matrix
XGB_confusion_matrix2 = confusion_matrix(test["Outcome"], y_result, labels=[1, 0])

In [65]:
# Old
XGB_confusion_matrix1

array([[ 3,  8],
       [ 3, 58]], dtype=int64)

In [66]:
# New
XGB_confusion_matrix2

array([[ 4,  7],
       [ 3, 58]], dtype=int64)

By tuning, we managed to gain one true positive.

Unfortunately, pharmaceutical data is hard to obtain and this particular one is of poor quality. The bad results from primary screening leads to the issue of whether this type of data should be used for virtual screening.